## A conversation with Andrew Ng
* **Transfer Learning** : 처음부터 많은 데이터로 오랜 시간동안 모델을 학습하는 것이 아니라, **open-source & pre-trained model**을 다운받아서 **그 가중치를 이용해 작은 데이터로도 학습**을 진행하는 것!
> 더 나은 CNN을 구축하는 데 도움됨 (내 적은 데이터셋에 없는 특징들을 발견했을 수 있기 때문에)    
  
* TF에서 모델 다운로드 -> 훈련 가능하도록 모델 set -> 다른 DNN 층들 잠그기 -> lower level 층만 가지고 retrain

## Understanding transfer learning: the concepts
* 이미지의 특징을 추출하는 수 많은 **convolution** 층들은 이미 학습된 가중치를 갖다 사용하고(**lock**), 그 다음의 **dense 층을 내 데이터로 retrain** 하면 된다!
* ex. **Image-Net** : 1.4 million images in a 1000 different classes (pre-trained model)

## Coding transfer learning from the inception mode

In [2]:
import os

from tensorflow.keras import layers
from tensorflow.keras import Model

!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (150, 150, 3),
                                include_top = False, # InceptionV3는 상단(출력층쪽)에 FC --> 무시하고 CNN으로 ㄱㄱ
                                weights = None) # don't use built-in weights

pre_trained_model.load_weights(local_weights_file)

# 각 층마다 lock 설정하기 (훈련하지 않을 층들)
for layer in pre_trained_model.layers:
  layer.trainable = False 

pre_trained_model.summary() # 수 많은 층들로 구성됨

--2021-02-23 06:24:03--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.188.128, 64.233.189.128, 108.177.97.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.188.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   154MB/s    in 0.5s    

2021-02-23 06:24:04 (154 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 150, 150, 3) 0                  

## Coding your own model with transferred features
* TensorFlow로 구성한 모델의 모든 뉴런층에는 이름이 있음. 그 중에서 'mixed7'이라는 이름의 층 가져오기
* 'mixed7' 이라는 이름을 갖는 층을 가져와서 사전 훈련된 신경망 모델의 마지막 층으로 지정 (3x3 대신 7x7이 필요하므로)
> 이 마지막 층의 출력을 입력으로 받아서 1차원으로 Flatten & Dense층 2개 추가

In [ ]:
last_layer = pre_trained_model.get_layer('mixed7') # output size가 7x7인 층 가져오기

last_output = last_layer.output


from tensorflow.keras.optimizers import RMSprop

# add layers to x
# 이런 방식으로도 tf.keras.layers API 사용 가능!
x = layers.Flatten()(last_output) # last_output층을 입력으로 받아서 Flatten층 추가
x = layers.Dense(1024, activation='relu')(x) # x층을 입력으로 받아서 Dense층 추가
x = layers.Dense(1, activation='sigmoid')(x) # 2개의 class 중 하나로 분류되어 나오는 출력 뉴런

# create model
model = Model(pre_trained_model.input, x) # 모델 생성, pre-trained 모델의 input정보와 방금 생성한 x층 정의 전달

# compile model
model.compile(optimizer = RMSprop(lr=0.0001),
              loss = 'binary_crossentropy',
              metrics = ['acc'])

# Add our data-augmentation parameters to ImageDataGenerator (Augmentation!)
# IDG 객체 생성
train_datagen = ImageDataGenerator(rescale=1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# load data and augmentation with IDG instance
train_generator = train_datagen.flow_from_directory(
    train_dir,
    batch_size = 20,
    class_mode = 'binary',
    target_size = (150, 150)
)

# train model with fit_generator
history = model.fit_generator(
    train_generator,
    validation_data = validation_generator,
    steps_per_epoch = 100,
    epochs = 100,
    validation_steps = 50,
    verbose = 2
)

In [ ]:
# 위의 결과, train_acc는 잘 진행되지만, val_acc는 큰 폭으로 왔다갔다하면서 정확도가 떨어지는 등 bad...

# 이 때 해결할 수 있는 방법 : 'dropout!'

### [ cf. augmentation with ImageDataGenerator ]
* **이미지를 사용할 때마다 임의로 변형**을 가함으로써 마치 훨씬 더 많은 이미지를 보고 공부하는 것과 같은 학습 효과를 낸다!
* 즉, **학습 과정에서 즉석해서 그때그때** 이미지를 회전 (rotation)시키는 등의 랜덤 전처리를 해준다!
> cf. 학습 : fit_generator()로 train_generator 객체를 학습시킴

## Exploring dropouts
* ImageDataGenerator를 이용한 augmentation과 transfer learning으로도 overfitting 해결 불가...
* *노드가 많아지거나, 층이 복잡해지거나, 데이터가 적으면 overfitting 위험 있음* (**overfitting 유발 요소들**)
* 이 때 **dropout**으로 해결!
> **dropout 층** : 랜덤하게 노드를 끔으로써 학습 데이터에만 지나치게 의존하여 학습되는 것을 방지!

In [ ]:
from tensorflow.keras.optimizers import RMSprop

x = layers.Flatten()(last_output) 
x = layers.Dense(1024, activation='relu')(x) 
x = layers.Dropout(0.2)(x)                     # add dropout layer (0.2 : 삭제할 비율)
x = layers.Dense(1, activation='sigmoid')(x) 

# create model
model = Model(pre_trained_model.input, x) 

# compile model
model.compile(optimizer = RMSprop(lr=0.0001),
              loss = 'binary_crossentropy',
              metrics = ['acc'])

## Exploring Transfer Learning with Inception